# ASTR 597b Final Project

### Matt Wilde



### Create a LSD DB for the CRTS Periodic Variable Catalog
### File consists of an ascii from:
http://nesssi.cacr.caltech.edu/DataRelease/Varcat.html

In [1]:
! head CatalinaVars.tbl

# Catalina_Surveys_ID   Numerical_ID  RA_(J2000)     Dec        V_(mag) Period_(days) Amplitude Number_Obs Var_Type
CSS_J000020.4+103118  1109001041232  00:00:20.41   +10:31:18.9    14.62   1.491758    2.39  223 2 
CSS_J000031.5-084652  1009001044997  00:00:31.50   -08:46:52.3    14.14   0.404185    0.12  163 1 
CSS_J000036.9+412805  1140001063366  00:00:36.94   +41:28:05.7    17.39   0.274627    0.73  158 1 
CSS_J000037.5+390308  1138001069849  00:00:37.55   +39:03:08.1    17.74   0.30691    0.23  219 1 
CSS_J000103.3+105724  1109001050739  00:01:03.37   +10:57:24.4    15.25   1.5837582    0.11  223 8 
CSS_J000103.4+395744  1140001014341  00:01:03.46   +39:57:44.5    15.51   1.9670131    0.14  158 1 
CSS_J000106.9+120610  1112001016471  00:01:06.96   +12:06:10.3    15.85   0.297318    0.11  205 1 
CSS_J000110.8+400521  1140001018252  00:01:10.89   +40:05:21.1    13.69   0.6606980    0.09  158 1 
CSS_J000131.5+324913  1132001052010  00:01:31.54   +32:49:13.1    14.71   13.049549    0.1

## Create a map file may need for ascii files?

In [2]:
! cat crts.map

crts_id 1
num_id 2
ra 3
dec 4
Vmag 5 
period 6
amplitude 7
num_obs 8
var_type 9

## Create a yaml schema

In [9]:
! cat crts_vsc.yaml

# Schema for CRTS variable star catalog, crts_vsc table
filters: {complevel: 5, complib: blosc}
schema:
  common:
    primary_key: crts_id
    spatial_keys: [ra, dec]
    columns:
    - [crts_id, u8]
    - [num_id, i8]
    - [ra, f8]
    - [dec, f8]
    - [Vmag, f4]
    - [period, f8]
    - [amplitude, f4]
    - [num_obs, i8]
    - [var_type, i4]

## Check env variables

In [4]:
! source /astro/users/mwilde/.bash_profile

In [5]:
! echo $NWORKERS

Strange that sourceing .bash_profile doesnt work

In [6]:
import os

os.environ['NWORKERS'] = '8'
os.environ['LSD_DB'] = '/astro/users/mwilde/tempdir'
os.environ['LSD_CACHEDIR'] = '/astro/users/mwilde/tempdir'
os.environ['LSD_TEMPDIR'] = '/astro/users/mwilde/tempdir'

In [7]:
! echo $NWORKERS

8


## Create the Table

In [10]:
 ! lsd-admin create table --schema crts_vsc.yaml crts_vsc --drop-existing

Table 'crts_vsc' dropped.

-------- committing 20160315193842.634457 [crts_vsc] ---------
[crts_vsc] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.12 sec
[crts_vsc] Updating neighbors: Already up to date.
[crts_vsc] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.11 sec
[crts_vsc] Updating stats: [0 el.]>  0.00 sec
[crts_vsc] Marking tablets read-only...
----------- success 20160315193842.634457 [crts_vsc] ---------

Table 'crts_vsc' created.


## Import the data

In [11]:
! lsd-import text --cols-file crts.map --hms ra,dec --skip-header 1 \
crts_vsc CatalinaVars.tbl

Importing from 1 pieces:
  ===> Imported CatalinaVars.tbl                                                       [1/1, 100.00%] +  47055/47055       47055 (0/0 min.)
done

-------- committing 20160315193853.282953 [crts_vsc] ---------
[crts_vsc] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.14 sec
[crts_vsc] Updating neighbors: [1 el.]::::::::::::::::::::[8 el.]++++++++++++++++++++>  0.60 sec
           Total 373968 cached objects in 8 cells
[crts_vsc] Updating tablet catalog: [256 el.]::::::::::::::::::::>  0.13 sec
[crts_vsc] Updating stats: [9 el.]::::::::::::::::::::>  0.02 sec [9 el.]::::::::::::::::::::>  0.04 sec
[crts_vsc] Marking tablets read-only...
----------- success 20160315193853.282953 [crts_vsc] ---------

